In [2]:
from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, average_precision_score, roc_curve, auc, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.utils.class_weight import compute_class_weight
import xgboost as xgb
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from scipy.stats import rankdata
from sklearn.svm import OneClassSVM
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA


In [3]:
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/DSA4263/'

df_original = pd.read_csv(file_path + 'original_cleaned_dataset.csv')
df_smote = pd.read_csv(file_path + 'smote_dataset.csv')
df_adasyn = pd.read_csv(file_path + 'adasyn_dataset.csv')
df_oversampled = pd.read_csv(file_path + 'oversampled_dataset.csv')
df_test = pd.read_csv(file_path + 'test_dataset.csv')

Mounted at /content/drive


In [4]:
# Dictionary of training datasets
datasets = {
    "Original": df_original,
    "SMOTE": df_smote,
    "ADASYN": df_adasyn,
    "60-40": df_oversampled,
    "Original + Weight": df_original,
    "60-40 + Weight": df_oversampled
}

# Split test data
X_test = df_test.drop(columns=['fraud_reported'])
y_test = df_test['fraud_reported']

# Define sample weight function
def compute_sample_weights(y):
    class_counts = y.value_counts()
    total = len(y)
    class_weight = {cls: total / (len(class_counts) * count) for cls, count in class_counts.items()}
    sample_weights = y.map(class_weight)
    return sample_weights


##Unsupervised Learning Models

- To ensure the model is able to predict in real time, unsupervise machine learning model will be crucial to flag out suspected fraud cases.
- We will do 2 models, namely Isolation Forest and DBSCAN
- Isolation Forest detects global outliers based on feature distributions.
- DBSCAN detects local density anomalies (points in sparse neighborhoods).

## Isolation Forest (IF)

In [5]:
# Store results
results_if = []

for name, df in datasets.items():
    print(f"\n=== Running Isolation Forest on {name} dataset ===")

    # Separate features and labels
    X_train = df.drop(columns=['fraud_reported'])
    y_train = df['fraud_reported']

    # Train Isolation Forest
    iso = IsolationForest(
        n_estimators=150,
        contamination='auto',
        max_samples='auto',
        random_state=4263
    )

    if "Weight" in name:
        sample_weights = compute_sample_weights(y_train)
        iso.fit(X_train, sample_weight=sample_weights)
    else:
        iso.fit(X_train)

    # Get anomaly scores & predictions
    iso_scores = -iso.decision_function(X_test)  # higher = more anomalous
    iso_preds = iso.predict(X_test)              # -1 = anomaly, 1 = normal
    iso_preds = np.where(iso_preds == -1, 1, 0)  # convert to 1=fraud, 0=normal

    # Evaluation metrics
    roc_auc = roc_auc_score(y_test, iso_scores)
    accuracy = accuracy_score(y_test, iso_preds)
    recall = recall_score(y_test, iso_preds)
    precision = precision_score(y_test, iso_preds)
    f1 = f1_score(y_test, iso_preds)

    # Store summary metrics
    results_if.append({
        'Dataset': name,
        'ROC AUC': roc_auc,
        'Accuracy': accuracy,
        'Recall': recall,
        'Precision': precision,
        'F1-score': f1
    })

    # === Detailed Evaluation Table ===
    print("\n📊 Test Set Evaluation:")
    cm = confusion_matrix(y_test, iso_preds)
    print("\nConfusion Matrix:")
    print(pd.DataFrame(
        cm,
        index=['Actual Non-Fraud (0)', 'Actual Fraud (1)'],
        columns=['Predicted Non-Fraud (0)', 'Predicted Fraud (1)']
    ))

    print("\nClassification Report:")
    print(classification_report(y_test, iso_preds, digits=4))




=== Running Isolation Forest on Original dataset ===


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_


📊 Test Set Evaluation:

Confusion Matrix:
                      Predicted Non-Fraud (0)  Predicted Fraud (1)
Actual Non-Fraud (0)                      150                    0
Actual Fraud (1)                           50                    0

Classification Report:
              precision    recall  f1-score   support

           0     0.7500    1.0000    0.8571       150
           1     0.0000    0.0000    0.0000        50

    accuracy                         0.7500       200
   macro avg     0.3750    0.5000    0.4286       200
weighted avg     0.5625    0.7500    0.6429       200


=== Running Isolation Forest on SMOTE dataset ===

📊 Test Set Evaluation:

Confusion Matrix:
                      Predicted Non-Fraud (0)  Predicted Fraud (1)
Actual Non-Fraud (0)                      149                    1
Actual Fraud (1)                           50                    0

Classification Report:
              precision    recall  f1-score   support

           0     0.7487    0.99

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_


📊 Test Set Evaluation:

Confusion Matrix:
                      Predicted Non-Fraud (0)  Predicted Fraud (1)
Actual Non-Fraud (0)                      150                    0
Actual Fraud (1)                           50                    0

Classification Report:
              precision    recall  f1-score   support

           0     0.7500    1.0000    0.8571       150
           1     0.0000    0.0000    0.0000        50

    accuracy                         0.7500       200
   macro avg     0.3750    0.5000    0.4286       200
weighted avg     0.5625    0.7500    0.6429       200


=== Running Isolation Forest on 60-40 dataset ===


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_


📊 Test Set Evaluation:

Confusion Matrix:
                      Predicted Non-Fraud (0)  Predicted Fraud (1)
Actual Non-Fraud (0)                      150                    0
Actual Fraud (1)                           50                    0

Classification Report:
              precision    recall  f1-score   support

           0     0.7500    1.0000    0.8571       150
           1     0.0000    0.0000    0.0000        50

    accuracy                         0.7500       200
   macro avg     0.3750    0.5000    0.4286       200
weighted avg     0.5625    0.7500    0.6429       200


=== Running Isolation Forest on Original + Weight dataset ===


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_


📊 Test Set Evaluation:

Confusion Matrix:
                      Predicted Non-Fraud (0)  Predicted Fraud (1)
Actual Non-Fraud (0)                      150                    0
Actual Fraud (1)                           50                    0

Classification Report:
              precision    recall  f1-score   support

           0     0.7500    1.0000    0.8571       150
           1     0.0000    0.0000    0.0000        50

    accuracy                         0.7500       200
   macro avg     0.3750    0.5000    0.4286       200
weighted avg     0.5625    0.7500    0.6429       200


=== Running Isolation Forest on 60-40 + Weight dataset ===

📊 Test Set Evaluation:

Confusion Matrix:
                      Predicted Non-Fraud (0)  Predicted Fraud (1)
Actual Non-Fraud (0)                      150                    0
Actual Fraud (1)                           50                    0

Classification Report:
              precision    recall  f1-score   support

           0     0.750

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_

In [6]:
# Combine results into a DataFrame
results_if_df = pd.DataFrame(results_if)
display(results_if_df)


,Dataset,ROC AUC,Accuracy,Recall,Precision,F1-score
0,Original,0.444933,0.750,0.0,0.0,0.0
1,SMOTE,0.374533,0.745,0.0,0.0,0.0
2,ADASYN,0.475333,0.750,0.0,0.0,0.0
3,60-40,0.454533,0.750,0.0,0.0,0.0
4,Original + Weight,0.444933,0.750,0.0,0.0,0.0
5,60-40 + Weight,0.454533,0.750,0.0,0.0,0.0


## DBSCAN


In [7]:
# Store results
results_dbscan = []

for name, df in datasets.items():
    print(f"\n=== Running DBSCAN on {name} dataset ===")

    # Separate features and labels
    X_train = df.drop(columns=['fraud_reported'])
    y_train = df['fraud_reported']


    # Train DBSCAN on training set
    # eps and min_samples need tuning
    dbscan = DBSCAN(
        eps=1.0,           # distance threshold
        min_samples=10,     # min points to form a cluster
        metric='euclidean'
    )
    dbscan.fit(X_train)

    # Predict anomalies on test set using nearest cluster distance
    # Since DBSCAN has no .predict(), we assign points outside training clusters as anomalies
    from sklearn.neighbors import NearestNeighbors

    nbrs = NearestNeighbors(n_neighbors=5).fit(X_train)
    distances, _ = nbrs.kneighbors(X_test)
    mean_dist = distances.mean(axis=1)

    # Higher distance → more likely anomaly
    # Threshold can be set as percentile of distances in training set
    threshold = np.percentile(distances.mean(axis=1), 75)  # top 25% distance
    db_preds = (mean_dist > threshold).astype(int)  # 1=fraud, 0=normal

    # Evaluation metrics
    roc_auc = roc_auc_score(y_test, mean_dist)
    accuracy = accuracy_score(y_test, db_preds)
    recall = recall_score(y_test, db_preds)
    precision = precision_score(y_test, db_preds)
    f1 = f1_score(y_test, db_preds)

    # Store summary metrics
    results_dbscan.append({
        'Dataset': name,
        'ROC AUC': roc_auc,
        'Accuracy': accuracy,
        'Recall': recall,
        'Precision': precision,
        'F1-score': f1
    })

    # === Detailed Evaluation Table ===
    print("\n📊 Test Set Evaluation:")
    cm = confusion_matrix(y_test, db_preds)
    print("\nConfusion Matrix:")
    print(pd.DataFrame(
        cm,
        index=['Actual Non-Fraud (0)', 'Actual Fraud (1)'],
        columns=['Predicted Non-Fraud (0)', 'Predicted Fraud (1)']
    ))

    print("\nClassification Report:")
    print(classification_report(y_test, db_preds, digits=4))



=== Running DBSCAN on Original dataset ===

📊 Test Set Evaluation:

Confusion Matrix:
                      Predicted Non-Fraud (0)  Predicted Fraud (1)
Actual Non-Fraud (0)                      117                   33
Actual Fraud (1)                           33                   17

Classification Report:
              precision    recall  f1-score   support

           0     0.7800    0.7800    0.7800       150
           1     0.3400    0.3400    0.3400        50

    accuracy                         0.6700       200
   macro avg     0.5600    0.5600    0.5600       200
weighted avg     0.6700    0.6700    0.6700       200


=== Running DBSCAN on SMOTE dataset ===

📊 Test Set Evaluation:

Confusion Matrix:
                      Predicted Non-Fraud (0)  Predicted Fraud (1)
Actual Non-Fraud (0)                      116                   34
Actual Fraud (1)                           34                   16

Classification Report:
              precision    recall  f1-score   suppor

In [8]:
# Combine results into a DataFrame
results_dbscan_df = pd.DataFrame(results_dbscan)
display(results_dbscan_df)

,Dataset,ROC AUC,Accuracy,Recall,Precision,F1-score
0,Original,0.602533,0.67,0.34,0.34,0.34
1,SMOTE,0.600667,0.66,0.32,0.32,0.32
2,ADASYN,0.594533,0.66,0.32,0.32,0.32
3,60-40,0.600400,0.65,0.30,0.30,0.30
4,Original + Weight,0.602533,0.67,0.34,0.34,0.34
5,60-40 + Weight,0.600400,0.65,0.30,0.30,0.30


### Now we do on the edited dataset, with the removal of the auto columns

In [9]:
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/DSA4263/'

df_original_edited = pd.read_csv(file_path + 'original_cleaned_dataset_edited.csv')
df_smote_edited = pd.read_csv(file_path + 'smote_dataset_edited.csv')
df_adasyn_edited = pd.read_csv(file_path + 'adasyn_dataset_edited.csv')
df_oversampled_edited = pd.read_csv(file_path + 'oversampled_dataset_edited.csv')
df_test_edited = pd.read_csv(file_path + 'test_dataset_edited.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# Dictionary of training datasets
datasets_edited = {
    "Original": df_original_edited,
    "SMOTE": df_smote_edited,
    "ADASYN": df_adasyn_edited,
    "60-40": df_oversampled_edited,
    "Original + Weight": df_original_edited,
    "60-40 + Weight": df_oversampled_edited
}

# Split test data
X_test = df_test_edited.drop(columns=['fraud_reported'])
y_test = df_test_edited['fraud_reported']

# Define sample weight function
def compute_sample_weights(y):
    class_counts = y.value_counts()
    total = len(y)
    class_weight = {cls: total / (len(class_counts) * count) for cls, count in class_counts.items()}
    sample_weights = y.map(class_weight)
    return sample_weights

# Isolation Forest


In [11]:
# Store results
results_if_edited = []

for name, df in datasets_edited.items():
    print(f"\n=== Running Isolation Forest on {name} dataset ===")

    # Separate features and labels
    X_train = df.drop(columns=['fraud_reported'])
    y_train = df['fraud_reported']

    # Train Isolation Forest
    iso = IsolationForest(
        n_estimators=150,
        contamination='auto',
        max_samples='auto',
        random_state=4263
    )

    if "Weight" in name:
        sample_weights = compute_sample_weights(y_train)
        iso.fit(X_train, sample_weight=sample_weights)
    else:
        iso.fit(X_train)

    # Get anomaly scores & predictions
    iso_scores = -iso.decision_function(X_test)  # higher = more anomalous
    iso_preds = iso.predict(X_test)              # -1 = anomaly, 1 = normal
    iso_preds = np.where(iso_preds == -1, 1, 0)  # convert to 1=fraud, 0=normal

    # Evaluation metrics
    roc_auc = roc_auc_score(y_test, iso_scores)
    accuracy = accuracy_score(y_test, iso_preds)
    recall = recall_score(y_test, iso_preds)
    precision = precision_score(y_test, iso_preds)
    f1 = f1_score(y_test, iso_preds)

    # Store summary metrics
    results_if_edited.append({
        'Dataset': name,
        'ROC AUC': roc_auc,
        'Accuracy': accuracy,
        'Recall': recall,
        'Precision': precision,
        'F1-score': f1
    })

    # === Detailed Evaluation Table ===
    print("\n📊 Test Set Evaluation:")
    cm = confusion_matrix(y_test, iso_preds)
    print("\nConfusion Matrix:")
    print(pd.DataFrame(
        cm,
        index=['Actual Non-Fraud (0)', 'Actual Fraud (1)'],
        columns=['Predicted Non-Fraud (0)', 'Predicted Fraud (1)']
    ))

    print("\nClassification Report:")
    print(classification_report(y_test, iso_preds, digits=4))




=== Running Isolation Forest on Original dataset ===

📊 Test Set Evaluation:

Confusion Matrix:
                      Predicted Non-Fraud (0)  Predicted Fraud (1)
Actual Non-Fraud (0)                       23                  127
Actual Fraud (1)                            8                   42

Classification Report:
              precision    recall  f1-score   support

           0     0.7419    0.1533    0.2541       150
           1     0.2485    0.8400    0.3836        50

    accuracy                         0.3250       200
   macro avg     0.4952    0.4967    0.3189       200
weighted avg     0.6186    0.3250    0.2865       200


=== Running Isolation Forest on SMOTE dataset ===

📊 Test Set Evaluation:

Confusion Matrix:
                      Predicted Non-Fraud (0)  Predicted Fraud (1)
Actual Non-Fraud (0)                       32                  118
Actual Fraud (1)                           13                   37

Classification Report:
              precision    recal

In [12]:
# Combine results into a DataFrame
results_if_edited_df = pd.DataFrame(results_if_edited)
display(results_if_edited_df)


,Dataset,ROC AUC,Accuracy,Recall,Precision,F1-score
0,Original,0.460667,0.325,0.84,0.248521,0.383562
1,SMOTE,0.397733,0.345,0.74,0.238710,0.360976
2,ADASYN,0.405733,0.315,0.78,0.236364,0.362791
3,60-40,0.434800,0.330,0.70,0.227273,0.343137
4,Original + Weight,0.460667,0.325,0.84,0.248521,0.383562
5,60-40 + Weight,0.434800,0.330,0.70,0.227273,0.343137


# DBSCAN

In [13]:
# Store results
results_dbscan_edited = []

for name, df in datasets_edited.items():
    print(f"\n=== Running DBSCAN on {name} dataset ===")

    # Separate features and labels
    X_train = df.drop(columns=['fraud_reported'])
    y_train = df['fraud_reported']

    # Train DBSCAN on training set
    # eps and min_samples need tuning
    dbscan = DBSCAN(
        eps=1.0,           # distance threshold
        min_samples=10,     # min points to form a cluster
        metric='euclidean'
    )
    dbscan.fit(X_train)

    # Predict anomalies on test set using nearest cluster distance
    # Since DBSCAN has no .predict(), we assign points outside training clusters as anomalies
    from sklearn.neighbors import NearestNeighbors

    nbrs = NearestNeighbors(n_neighbors=5).fit(X_train)
    distances, _ = nbrs.kneighbors(X_test)
    mean_dist = distances.mean(axis=1)

    # Higher distance → more likely anomaly
    # Threshold can be set as percentile of distances in training set
    threshold = np.percentile(distances.mean(axis=1), 75)  # top 25% distance
    db_preds = (mean_dist > threshold).astype(int)  # 1=fraud, 0=normal

    # Evaluation metrics
    roc_auc = roc_auc_score(y_test, mean_dist)
    accuracy = accuracy_score(y_test, db_preds)
    recall = recall_score(y_test, db_preds)
    precision = precision_score(y_test, db_preds)
    f1 = f1_score(y_test, db_preds)

    # Store summary metrics
    results_dbscan_edited.append({
        'Dataset': name,
        'ROC AUC': roc_auc,
        'Accuracy': accuracy,
        'Recall': recall,
        'Precision': precision,
        'F1-score': f1
    })

    # === Detailed Evaluation Table ===
    print("\n📊 Test Set Evaluation:")
    cm = confusion_matrix(y_test, db_preds)
    print("\nConfusion Matrix:")
    print(pd.DataFrame(
        cm,
        index=['Actual Non-Fraud (0)', 'Actual Fraud (1)'],
        columns=['Predicted Non-Fraud (0)', 'Predicted Fraud (1)']
    ))

    print("\nClassification Report:")
    print(classification_report(y_test, db_preds, digits=4))



=== Running DBSCAN on Original dataset ===

📊 Test Set Evaluation:

Confusion Matrix:
                      Predicted Non-Fraud (0)  Predicted Fraud (1)
Actual Non-Fraud (0)                      117                   33
Actual Fraud (1)                           33                   17

Classification Report:
              precision    recall  f1-score   support

           0     0.7800    0.7800    0.7800       150
           1     0.3400    0.3400    0.3400        50

    accuracy                         0.6700       200
   macro avg     0.5600    0.5600    0.5600       200
weighted avg     0.6700    0.6700    0.6700       200


=== Running DBSCAN on SMOTE dataset ===

📊 Test Set Evaluation:

Confusion Matrix:
                      Predicted Non-Fraud (0)  Predicted Fraud (1)
Actual Non-Fraud (0)                      115                   35
Actual Fraud (1)                           35                   15

Classification Report:
              precision    recall  f1-score   suppor

In [14]:
# Combine results into a DataFrame
results_dbscan_edited_df = pd.DataFrame(results_dbscan_edited)
display(results_dbscan_edited_df)

,Dataset,ROC AUC,Accuracy,Recall,Precision,F1-score
0,Original,0.622800,0.67,0.34,0.34,0.34
1,SMOTE,0.606133,0.65,0.30,0.30,0.30
2,ADASYN,0.600933,0.65,0.30,0.30,0.30
3,60-40,0.613200,0.67,0.34,0.34,0.34
4,Original + Weight,0.622800,0.67,0.34,0.34,0.34
5,60-40 + Weight,0.613200,0.67,0.34,0.34,0.34


# L1 regularisation to keep only the important features


In [15]:
# Separate features and labels
X_train = df_original_edited.drop(columns=['fraud_reported'])
y_train = df_original_edited['fraud_reported']
X_test = df_test_edited.drop(columns=['fraud_reported'])
y_test = df_test_edited['fraud_reported']

lasso = LogisticRegression(
    penalty='l1',
    solver='liblinear',
    class_weight='balanced',  # handle imbalance
    random_state=42,
    max_iter=1000
)
lasso.fit(X_train, y_train)

# Identify non-zero features (kept by L1)
selected_features = X_train.columns[lasso.coef_[0] != 0]
print(f"Selected {len(selected_features)} features:")
print(selected_features.tolist())


Selected 52 features:
['months_as_customer', 'age', 'policy_csl', 'policy_deductable', 'policy_annual_premium', 'umbrella_limit', 'insured_zip', 'insured_sex', 'insured_education_level', 'capital-gains', 'incident_hour_of_the_day', 'number_of_vehicles_involved', 'bodily_injuries', 'witnesses', 'total_claim_amount', 'injury_claim', 'property_claim', 'policy_state_IL', 'insured_occupation_Manual', 'insured_occupation_Professional', 'insured_occupation_Sales/Service', 'insured_hobbies_Extreme', 'insured_hobbies_Games', 'insured_hobbies_Leisure', 'insured_hobbies_Outdoor', 'insured_hobbies_Sports', 'insured_relationship_husband', 'insured_relationship_not-in-family', 'insured_relationship_own-child', 'insured_relationship_unmarried', 'insured_relationship_wife', 'incident_type_Parked Car', 'incident_type_Single Vehicle Collision', 'collision_type_Front Collision', 'collision_type_Side Collision', 'incident_severity_Major Damage', 'incident_severity_Minor Damage', 'incident_severity_Trivial

# Isolation Forest

In [16]:
# Initialize Isolation Forest
iso = IsolationForest(
    n_estimators=150,
    contamination=0.25,
    random_state=42
)

# Fit on training data
iso.fit(X_train[selected_features])

# Predict on test data
iso_scores = -iso.decision_function(X_test[selected_features])  # higher = more anomalous
iso_preds = iso.predict(X_test[selected_features])
iso_preds = np.where(iso_preds == -1, 1, 0)  # convert: -1→fraud(1), 1→non-fraud(0)


# Evaluation metrics
roc_auc = roc_auc_score(y_test, iso_scores)
accuracy = accuracy_score(y_test, iso_preds)
recall = recall_score(y_test, iso_preds)
precision = precision_score(y_test, iso_preds)
f1 = f1_score(y_test, iso_preds)


# === Detailed Evaluation Table ===
print("\n📊 Test Set Evaluation:")
cm = confusion_matrix(y_test, iso_preds)
print("\nConfusion Matrix:")
print(pd.DataFrame(
        cm,
        index=['Actual Non-Fraud (0)', 'Actual Fraud (1)'],
        columns=['Predicted Non-Fraud (0)', 'Predicted Fraud (1)']
    ))

print("\nClassification Report:")
print(classification_report(y_test, iso_preds, digits=4))



📊 Test Set Evaluation:

Confusion Matrix:
                      Predicted Non-Fraud (0)  Predicted Fraud (1)
Actual Non-Fraud (0)                      106                   44
Actual Fraud (1)                           30                   20

Classification Report:
              precision    recall  f1-score   support

           0     0.7794    0.7067    0.7413       150
           1     0.3125    0.4000    0.3509        50

    accuracy                         0.6300       200
   macro avg     0.5460    0.5533    0.5461       200
weighted avg     0.6627    0.6300    0.6437       200



# Hyperparameter Tuning on IF

In [17]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_samples': ['auto', 0.8, 1.0],
    'contamination': [0.01, 0.05, 0.1, 0.25],
    'max_features': [0.6, 0.8, 1.0]
}

results = []

for n in param_grid['n_estimators']:
    for ms in param_grid['max_samples']:
        for c in param_grid['contamination']:
            for mf in param_grid['max_features']:

                iso = IsolationForest(
                    n_estimators=n,
                    max_samples=ms,
                    contamination=c,
                    max_features=mf,
                    random_state=42
                )
                iso.fit(X_train[selected_features])

                # Predict
                iso_scores = -iso.decision_function(X_test[selected_features])
                iso_preds = np.where(iso.predict(X_test[selected_features]) == -1, 1, 0)

                # Evaluate
                roc_auc = roc_auc_score(y_test, iso_scores)
                f1 = f1_score(y_test, iso_preds)
                recall = recall_score(y_test, iso_preds)
                precision = precision_score(y_test, iso_preds)
                accuracy = accuracy_score(y_test, iso_preds)

                results.append({
                    'n_estimators': n,
                    'max_samples': ms,
                    'contamination': c,
                    'max_features': mf,
                    'roc_auc': roc_auc,
                    'f1': f1,
                    'recall': recall,
                    'precision': precision,
                    'accuracy': accuracy
                })

# Convert results to DataFrame
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by='f1', ascending=False)
print("Top Results:")
print(results_df.head(10))

Top Results:
     n_estimators max_samples  contamination  max_features   roc_auc  \
22            100         0.8           0.25           0.8  0.547733   
69            200         1.0           0.25           0.6  0.518000   
23            100         0.8           0.25           1.0  0.527067   
34            100         1.0           0.25           0.8  0.541867   
70            200         1.0           0.25           0.8  0.531200   
94            300         0.8           0.25           0.8  0.551467   
58            200         0.8           0.25           0.8  0.545333   
106           300         1.0           0.25           0.8  0.531333   
105           300         1.0           0.25           0.6  0.517067   
47            200        auto           0.25           1.0  0.542133   

           f1  recall  precision  accuracy  
22   0.402985    0.54   0.321429     0.600  
69   0.366412    0.48   0.296296     0.585  
23   0.360902    0.48   0.289157     0.575  
34   0.359712 

# DBSCAN

In [18]:
# Separate features and labels
X_train = X_train[selected_features]
X_test = X_test[selected_features]

# Train DBSCAN on training set
# eps and min_samples need tuning
dbscan = DBSCAN(
    eps=1.0,           # distance threshold
    min_samples=10,     # min points to form a cluster
    metric='euclidean'
)
dbscan.fit(X_train)

# Predict anomalies on test set using nearest cluster distance
# Since DBSCAN has no .predict(), we assign points outside training clusters as anomalies
from sklearn.neighbors import NearestNeighbors

nbrs = NearestNeighbors(n_neighbors=5).fit(X_train)
distances, _ = nbrs.kneighbors(X_test)
mean_dist = distances.mean(axis=1)

# Higher distance → more likely anomaly
# Threshold can be set as percentile of distances in training set
threshold = np.percentile(distances.mean(axis=1), 75)  # top 25% distance
db_preds = (mean_dist > threshold).astype(int)  # 1=fraud, 0=normal

# Evaluation metrics
roc_auc = roc_auc_score(y_test, mean_dist)
accuracy = accuracy_score(y_test, db_preds)
recall = recall_score(y_test, db_preds)
precision = precision_score(y_test, db_preds)
f1 = f1_score(y_test, db_preds)

# === Detailed Evaluation Table ===
print("\n📊 Test Set Evaluation:")
cm = confusion_matrix(y_test, db_preds)
print("\nConfusion Matrix:")
print(pd.DataFrame(
    cm,
    index=['Actual Non-Fraud (0)', 'Actual Fraud (1)'],
    columns=['Predicted Non-Fraud (0)', 'Predicted Fraud (1)']
))

print("\nClassification Report:")
print(classification_report(y_test, db_preds, digits=4))



📊 Test Set Evaluation:

Confusion Matrix:
                      Predicted Non-Fraud (0)  Predicted Fraud (1)
Actual Non-Fraud (0)                      118                   32
Actual Fraud (1)                           32                   18

Classification Report:
              precision    recall  f1-score   support

           0     0.7867    0.7867    0.7867       150
           1     0.3600    0.3600    0.3600        50

    accuracy                         0.6800       200
   macro avg     0.5733    0.5733    0.5733       200
weighted avg     0.6800    0.6800    0.6800       200



# Hyperparameter Tuning on DBSCAN

In [21]:
best_result = {
    'eps': None,
    'min_samples': None,
    'n_neighbors': None,
    'threshold_percentile': None,
    'f1': -1,
    'recall': -1
}

# Define parameter grid
eps_values = [0.5, 0.8, 1.0, 1.2, 1.5]
min_samples_values = [5, 10, 15, 20]
n_neighbors_values = [3, 5, 7, 10]
threshold_percentiles = [70, 75, 80, 85, 90, 95]

# Start grid search
for eps in eps_values:
    for min_samples in min_samples_values:
        for n_neighbors in n_neighbors_values:
            try:
                # Train DBSCAN
                dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric='euclidean')
                dbscan.fit(X_train)

                # Compute nearest neighbor distances
                nbrs = NearestNeighbors(n_neighbors=n_neighbors).fit(X_train)
                distances, _ = nbrs.kneighbors(X_test)
                mean_dist = distances.mean(axis=1)

                for pct in threshold_percentiles:
                    threshold = np.percentile(distances.mean(axis=1), pct)
                    db_preds = (mean_dist > threshold).astype(int)

                    # Evaluate metrics
                    f1 = f1_score(y_test, db_preds)
                    recall = recall_score(y_test, db_preds)

                    if (f1 > best_result['f1']) or (f1 == best_result['f1'] and recall > best_result['recall']):
                        best_result.update({
                            'eps': eps,
                            'min_samples': min_samples,
                            'n_neighbors': n_neighbors,
                            'threshold_percentile': pct,
                            'f1': f1,
                            'recall': recall,
                        })
            except Exception as e:
                print(f"⚠️ Skipped eps={eps}, min_samples={min_samples}, n_neighbors={n_neighbors} due to error: {e}")

print("\nBest Parameters Found:")
for k, v in best_result.items():
    print(f"{k}: {v}")

# === Final Model Using Best Params ===
best_dbscan = DBSCAN(eps=best_result['eps'], min_samples=best_result['min_samples'])
best_dbscan.fit(X_train)

nbrs = NearestNeighbors(n_neighbors=best_result['n_neighbors']).fit(X_train)
distances, _ = nbrs.kneighbors(X_test)
mean_dist = distances.mean(axis=1)
threshold = np.percentile(distances.mean(axis=1), best_result['threshold_percentile'])
db_preds = (mean_dist > threshold).astype(int)

# Final evaluation
print("\nFinal Evaluation on Test Set:")
cm = confusion_matrix(y_test, db_preds)
print(pd.DataFrame(cm,
                   index=['Actual Non-Fraud (0)', 'Actual Fraud (1)'],
                   columns=['Predicted Non-Fraud (0)', 'Predicted Fraud (1)']))
print("\nClassification Report:")
print(classification_report(y_test, db_preds, digits=4))



Best Parameters Found:
eps: 0.5
min_samples: 5
n_neighbors: 10
threshold_percentile: 70
f1: 0.38181818181818183
recall: 0.42

Final Evaluation on Test Set:
                      Predicted Non-Fraud (0)  Predicted Fraud (1)
Actual Non-Fraud (0)                      111                   39
Actual Fraud (1)                           29                   21

Classification Report:
              precision    recall  f1-score   support

           0     0.7929    0.7400    0.7655       150
           1     0.3500    0.4200    0.3818        50

    accuracy                         0.6600       200
   macro avg     0.5714    0.5800    0.5737       200
weighted avg     0.6821    0.6600    0.6696       200

